In [129]:
%pylab inline
import re
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [4]:
!ls \data

20160116S1-FORZA_16_Asics_Black.txt 20160116S2-LAVA_North_16_Adidas.txt
20160116S1-LAVA_North_16_Adidas.txt 20160116S2-SCVC_16_1.txt
20160116S1-SCVC_16_1.txt            20160116S3-SCVC_16_1.txt
20160116S2-FORZA_16_Asics_Black.txt


In [1040]:
skillList = ['Sv','Batt', 'A', 'S','D','B','BA', 'P', 'H']

data = !cat data/20160116S1-LAVA_North_16_Adidas.txt | grep '^[0-9].*'

lineup = !cat data/20160116S1-LAVA_North_16_Adidas.txt | grep '^lineup.*' | cut -d' ' -f2
lineup = lineup[0].split('|')

j=0
rotationStats = []
server = ''
playerSkillResults = []

for i in range(0,len(data)):
    #get data for this play
    play_data = data[i].split('|')
    begin_score = play_data[0]
    
    #get resulting score from play
    if i+1 < len(data):
        result_score = data[i+1].split('|')[0]
        del play_data[0]
    else:
        result_score = data[i].split('|')[0]
        
    #determine whether the lineup has changed and change it if you need to
    if any('sub' in s for s in play_data):
        subs = [s for s in play_data if 'sub' in s]
        for c in subs:
            lineup_change = c.replace('sub ','').split('/')
            lineup = [lineup_change[0] if x == lineup_change[1] else x for x in lineup]           
    
    #determine whether or not we are in a new rotation
    if any('Sv' in s for s in play_data):
        this = [s for s in play_data if 'Sv' in s]
        if this:
            server = re.findall(r'\d+', this[0])

        try:
            last_server
        except NameError:
            last_server = server
        
        #print server, last_server
        if server != last_server :
            j += 1
        last_server = server

        if j%6 == 0:
            rotation = str(6)
        else:
            rotation = str(j%6)
    
    #Account for setting assists
    print [s for s in play_data if 'Ak' in s]

    #get active player list for play (i.e. players who touched the ball on the play)
    active = []
    for a in [s for s in play_data if 'sub' not in s]:
        all_digits = re.findall(r'\d+', a)
        if len(all_digits) > 0 and i+1 < len(data):
            active.append(all_digits[0])
            player = all_digits[0]
            skill = str(re.findall(r'[A-Z]+', a)).strip("['']")
            if len(all_digits) > 1:
                result = all_digits[1]
            else:
                result = str(re.findall(r'[\+\-aek]+', a)).strip("['']")
        playerSkillResults.append([player, skill , result, 1])
    #print active
    
    if i+1 < len(data):
        home_begin_score = int(begin_score.split('-')[0])
        opponent_begin_score = int(begin_score.split('-')[1])
        home_result_score = int(result_score.split('-')[0])
        opponent_result_score = int(result_score.split('-')[1])
        point = home_result_score - home_begin_score - opponent_result_score + opponent_begin_score 
        
        rotationStats.append([begin_score, rotation, play_data, result_score, point, lineup, list(set(active))])
        
df = pd.DataFrame.from_records(rotationStats)
df.columns = ['start_score', 'rotation', 'play_action','result_score','point', 'lineup', 'active']
df

[]
[]
[]
[]
[]
[]
[]
[]
['6Ak']
[]
[]
[]
[]
['9Ak']
['11Ak']
[]
[]
['15Ak']
[]
[]
['11Ak']
[]
['15Ak']
[]
[]
[]
[]
[]
['2Ak']
['11Ak']
[]
[]
[]
['9Ak']
[]
[]
[]
[]
[]
[]
[]
['15Ak']
[]
[]
[]
[]
[]
['15Ak']
[]


,start_score,rotation,play_action,result_score,point,lineup,active
0,0-0,6,[2SV],1-0,1,"[2, 6, 9, 23, 15, 4]",[2]
1,1-0,6,"[2SV, 6BAT]",1-1,-1,"[2, 6, 9, 23, 15, 4]","[2, 6]"
2,1-1,6,[15P0],1-2,-1,"[2, 6, 9, 23, 15, 4]",[15]
3,1-2,6,"[15P3, 2S+, 23Ae]",1-3,-1,"[2, 6, 9, 23, 15, 4]","[2, 15, 23]"
4,1-3,6,[9P0],1-4,-1,"[2, 6, 9, 23, 15, 4]",[9]
5,1-4,6,"[4P3, 2S, 6A, 23De]",1-5,-1,"[2, 6, 9, 23, 15, 4]","[23, 2, 4, 6]"
6,1-5,6,"[4P3, 2S, 9A, 6D, 2S, 23Ae]",1-6,-1,"[2, 6, 9, 23, 15, 4]","[9, 23, 2, 4, 6]"
7,1-6,6,"[6P3, 2S+, 9Ae]",1-7,-1,"[2, 6, 9, 23, 15, 4]","[9, 2, 6]"
8,1-7,6,"[15P3, 2S+, 6Ak]",2-7,1,"[2, 6, 9, 23, 15, 4]","[2, 15, 6]"
9,2-7,6,"[sub 3/6, 3SV, 15B, 9BA]",3-7,1,"[2, 3, 9, 23, 15, 4]","[9, 3, 15]"


In [1015]:
result_df = pd.DataFrame.from_records(playerSkillResults, columns=('player', 'skill', 'result', 'total'))
result_df = result_df.groupby(['player','skill', 'result']).count()
pd.options.display.multi_sparse = False
result_df = result_df.reset_index()

In [1016]:
#HITTING
attackDF = result_df[['player', 'result', 'total']][result_df['skill'] == 'A']
total = attackDF.groupby('player').sum()
kills = attackDF[attackDF['result'] == 'k'].groupby('player').sum()
kills.columns = ['kills']
error =  attackDF[attackDF['result'] == 'e'].groupby('player').sum()
error.columns = ['error']
good = attackDF[attackDF['result'] == '+'].groupby('player').sum()
good.columns = ['good']
bad = attackDF[attackDF['result'] == '+'].groupby('player').sum()
bad.columns = ['bad']
neutral = attackDF[attackDF['result'] == ''].groupby('player').sum()
neutral.columns = ['neutral']

attack_stats = pd.concat([kills, error, good, bad, neutral, total], axis='1', join='outer')
attack_stats = attack_stats.fillna(0)
attack_stats['Attack Pct'] = (attack_stats.kills - attack_stats.error) / attack_stats.total

print 'ATTACK', attack_stats.sort_values('Attack Pct', ascending=False), '\n'

ATTACK     kills  error  good  bad  neutral  total  Attack Pct
11      3      0     0    0        0      3    1.000000
2       1      0     0    0        0      1    1.000000
15      5      0     0    0        4      9    0.555556
6       1      1     0    0        4      6    0.000000
9       2      2     0    0        1      5    0.000000
14      0      1     0    0        1      2   -0.500000
23      0      3     0    0        2      5   -0.600000 



In [1017]:
#SERVING
serveDF = result_df[['player', 'result', 'total']][result_df['skill'] == 'SV']
total = serveDF.groupby('player').sum()
aces = serveDF[serveDF['result'] == 'a'].groupby('player').sum()
aces.columns = ['aces']
error =  serveDF[serveDF['result'] == 'e'].groupby('player').sum()
error.columns = ['error']
good = serveDF[serveDF['result'] == '+'].groupby('player').sum()
good.columns = ['good']
neutral = serveDF[serveDF['result'] == ''].groupby('player').sum()
neutral.columns = ['neutral']

serve_stats = pd.concat([aces, error, good, neutral, total], axis='1', join='outer')
serve_stats = serve_stats.fillna(0)
serve_stats['Serve Pct'] = (serve_stats.aces - serve_stats.error) / serve_stats.total

print 'SERVE', serve_stats.sort_values('Serve Pct', ascending=False), '\n'

SERVE     aces  error  good  neutral  total  Serve Pct
14     2      0     2        4      8   0.250000
3      1      0     0        5      6   0.166667
2      0      0     0        3      3   0.000000
4      0      0     0        1      1   0.000000
9      1      1     0        2      4   0.000000
15     0      2     1        1      4  -0.500000 



In [1018]:
#PASSING
passDF = result_df[['player', 'result', 'total']][result_df['skill'] == 'P']
total = passDF.groupby('player').sum()
error = passDF[passDF['result'] == '0'].groupby('player').sum()
error.columns = ['0']
ones =  passDF[passDF['result'] == '1'].groupby('player').sum()
ones.columns = ['1']
twos = passDF[passDF['result'] == '2'].groupby('player').sum()
twos.columns = ['2']
threes = passDF[passDF['result'] == '3'].groupby('player').sum()
threes.columns = ['3']

passing_stats = pd.concat([error, ones, twos, threes, total], axis='1', join='outer')
passing_stats = passing_stats.fillna(0)
passing_stats['Passing Weighted'] = ((passing_stats['0']*0 )+ (passing_stats['1']*1) + (passing_stats['2']*2 )
                                + (passing_stats['3']*3)) / (passing_stats['total'])

print 'PASS', passing_stats.sort_values('Passing Weighted', ascending=False), '\n'

PASS     0  1  2  3  total  Passing Weighted
3   0  1  1  6      8          2.625000
15  1  0  0  5      6          2.500000
6   0  1  0  1      2          2.000000
4   1  3  4  4     12          1.916667
9   1  0  0  0      1          0.000000 



In [1025]:
#SETTING
settingDF = result_df[['player', 'result', 'total']][result_df['skill'] == 'S']
set_total = settingDF.groupby('player').sum()
good = settingDF[settingDF['result'] == '+'].groupby('player').sum()
good.columns = ['+']
bad = settingDF[settingDF['result'] == '-'].groupby('player').sum()
bad.columns = ['-']
error = settingDF[settingDF['result'] == 'e'].groupby('player').sum()
error.columns = ['error']

setting_stats = pd.concat([good, bad, error, total], axis='1', join='outer')
setting_stats = setting_stats.fillna(0)
#attack_stats['Attack Pct'] = (attack_stats.kills - attack_stats.error) / attack_stats.total
setting_stats

,+,-,error,total
2,8,0,0,26
3,0,0,0,1
4,0,0,0,2


In [784]:
positivePointDict = {}
totalPointsPlayed = {}
totalActivePointsPlayed = {}
positiveActivePointsPlayed = {}

for idx, col in df[['point','lineup']].iterrows():
    for a in col[1]:
        if a in positivePointDict.keys():
            positivePointDict[a] += col[0]
            totalPointsPlayed[a] += 1
        else:
            positivePointDict[a] = col[0]
            totalPointsPlayed[a] = 1
            
for idx, col in df[['point','active']].iterrows():
    for a in col[1]:
        if a in positiveActivePointsPlayed.keys():
            positiveActivePointsPlayed[a] += col[0]
            totalActivePointsPlayed[a] += 1
        else:
            positiveActivePointsPlayed[a] = col[0]
            totalActivePointsPlayed[a] = 1

print positivePointDict
print totalPointsPlayed

print totalActivePointsPlayed
print positiveActivePointsPlayed

{'11': 10, '15': 2, '14': 11, '23': -9, '3': 9, '2': 2, '4': -8, '6': -7, '9': 2}
{'11': 18, '15': 48, '14': 25, '23': 23, '3': 27, '2': 48, '4': 30, '6': 21, '9': 48}
{'11': 7, '15': 14, '14': 11, '23': 6, '3': 13, '2': 29, '4': 14, '6': 9, '9': 14}
{'11': 5, '15': 4, '14': 5, '23': -6, '3': 5, '2': -1, '4': -6, '6': -5, '9': 0}


In [237]:
roster = {25: 'Sophia', 14: 'Sabrina', 9: 'Olivia', 11: 'Morgan', 5: 'Lindsey', 15: 'Katie', 7: 'Mikayla', 4: 'Ellie', 2: 'Jordan', 6:'Emma', 3:'Lexi', 23:'Mekenzi'}
